In [1]:
import cov_3X2
import lsst_utils
from cov_3X2 import *
from lsst_utils import *
from scipy.stats import norm,mode,skew,kurtosis,percentileofscore
import seaborn as sns
colors=sns.color_palette()
import importlib
import matplotlib.pyplot as plt
import window_utils
importlib.reload(window_utils)
from window_utils import *
import tracer_utils
from tracer_utils import *
import treecorr
from wigner_transform import *

import logging
logger = logging.getLogger()
logger.setLevel(logging.WARNING)

%matplotlib inline
%config InlineBackend.figure_formats = ['svg']

In [2]:
from distributed import LocalCluster
from dask.distributed import Client  # we already had this above
#http://distributed.readthedocs.io/en/latest/_modules/distributed/worker.html
LC=LocalCluster(n_workers=4,processes=False,memory_limit='100gb',threads_per_worker=1,
                local_dir='/home/chienhal/jh/tmp/NGL-worker',
               )#memory_spill_fraction=.99,
#                memory_monitor_interval='2000ms')
client=Client(LC,diagnostics_port=8787)

/home/chienhal/anaconda3/lib/python3.6/site-packages/distributed/dashboard/core.py:74: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


In [3]:
client

Client Scheduler: inproc://10.1.2.211/122665/1 Dashboard: http://localhost:40621/status,Cluster Workers: 4 Cores: 4 Memory: 209.72 GB


In [4]:
#setup parameters
lmax_cl=1000
lmin_cl=0
l0=np.arange(lmin_cl,lmax_cl)
lmin_cl_Bins=lmin_cl+10
lmax_cl_Bins=lmax_cl-10
Nl_bins=40
l_bins=np.int64(np.logspace(np.log10(lmin_cl_Bins),np.log10(lmax_cl_Bins),Nl_bins))
# l_bins=np.int64(np.linspace(lmin_cl_Bins,lmax_cl_Bins,Nl_bins))
lb=(l_bins[1:]+l_bins[:-1])*.5  ##bin centers
l=l0 #np.unique(np.int64(np.logspace(np.log10(lmin_cl),np.log10(lmax_cl),Nl_bins*20))) #if we want to use fewer ell

do_cov=False
bin_cl=True
SSV_cov=True
tidal_SSV_cov=False
do_xi=False

use_window=True
store_win=True
f_sky=0.3
n_source_bins=1 ## 1 z-bins
sigma_gamma=0.3944/np.sqrt(2.)  #*2**0.25
nside=1024

corr_ggl=('galaxy','kappa')
corr_gg=('galaxy','galaxy')
corr_kk=('kappa','kappa') #or kappa 
corrs = [corr_kk, corr_ggl, corr_gg]

In [5]:
zl_bin=lsst_source_tomo_bins(ns=10,use_window=use_window,nbins=n_source_bins,
                         f_sky=f_sky,nside=nside,unit_win=True)

zs_bin=lsst_source_tomo_bins(ns=30,use_window=use_window,
                              f_sky=f_sky,nbins=n_source_bins,nside=nside,unit_win=True)


ns0:  27.0


./cov_utils.py:60: RuntimeWarning: invalid value encountered in true_divide
  Win0=2*jn(1,l_th)/l_th
./angular_power_spectra.py:23: RuntimeWarning: divide by zero encountered in true_divide
  self.cl_f=(l+0.5)**2/(l*(l+1.)) # cl correction from Kilbinger+ 2017
/home/chienhal/anaconda3/lib/python3.6/site-packages/astropy/units/quantity.py:466: RuntimeWarning: divide by zero encountered in true_divide
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)


cl dict done
ns0:  27.0
cl dict done


In [6]:
# zs_bin[0].SN['shear']
zs_bin[0].keys()

dict_keys(['z', 'dz', 'nz', 'ns', 'W', 'pz', 'pzdz', 'Norm', 'b1', 'AI', 'AI_z', 'mag_fact', 'lm', 'Gkernel', 'Gkernel_int', 'gkernel', 'gkernel_int', 'kernel_int', 'window', 'window_alm', 'window_cl'])

In [7]:
th_min=1./60
th_max=600./60
n_th_bins=20
th_bins=np.logspace(np.log10(th_min),np.log10(th_max),n_th_bins+1)
th=np.logspace(np.log10(th_min*0.98),np.log10(1),n_th_bins*30)
th2=np.linspace(1,th_max*1.02,n_th_bins*30)
th=np.unique(np.sort(np.append(th,th2)))
thb=np.sqrt(th_bins[1:]*th_bins[:-1])

l0_win=np.arange(lmax_cl)
WT_L_kwargs={'l': l0_win,'theta': th*d2r,'m1_m2':[(2,2),(2,-2),(0,2),(2,0),(0,0)]}
%time WT_L=wigner_transform(**WT_L_kwargs)

bin_xi=True

./wigner_functions.py:38: RuntimeWarning: invalid value encountered in true_divide
  d_mat/=np.sqrt(binom(k+b,b))
./wigner_functions.py:38: RuntimeWarning: divide by zero encountered in true_divide
  d_mat/=np.sqrt(binom(k+b,b))
./wigner_functions.py:38: RuntimeWarning: divide by zero encountered in true_divide
  d_mat/=np.sqrt(binom(k+b,b))


CPU times: user 2.13 s, sys: 1.16 s, total: 3.29 s
Wall time: 41.8 s


In [ ]:
## 
# wigner_files = {0: '/home/chienhal/jh/sklens_wig3j/skylens_wig3j/dask_wig3j_l5000_w500_0_asym50.zarr',
#                 2: '/home/chienhal/jh/sklens_wig3j/skylens_wig3j/dask_wig3j_l5000_w500_2_asym50.zarr'}
kappa_win=cov_3X2(zk_bins=zs_bin,do_cov=do_cov,bin_cl=bin_cl,l_bins=l_bins,l=l0, zg_bins=zl_bin,
            use_window=True,store_win=store_win,window_lmax=400,corrs=corrs,
            SSV_cov=SSV_cov,tidal_SSV_cov=tidal_SSV_cov,f_sky=f_sky,
            HT=WT_L,bin_xi=bin_xi,theta_bins=th_bins,do_xi=do_xi)

In [ ]:
# %time clG_win=kappa_win.cl_tomo(corrs=corrs) 
# %time cl0_win=clG_win['stack'].compute() #stack?

# if do_xi:
#     %time xiWG_L=kappa_win.xi_tomo()
#     %time xiW_L=xiWG_L['stack'].compute()

In [ ]:
# clG0=kappa_win.cl_tomo() 
# cl0 = (clG0['cl'][corr_kk][(0,0)].compute(), clG0['cl'][corr_kk][(0,1)].compute(), clG0['cl'][corr_kk][(1,1)].compute())

# np.random.seed(12345)
# cl_map = hp.synfast(cl0,nside=nside,new=False,pol=False)

In [ ]:
clG0=kappa_win.cl_tomo() 
cl0 = (clG0['cl'][corr_kk][(0,0)].compute(), clG0['cl'][corr_ggl][(0,0)].compute(), clG0['cl'][corr_gg][(0,0)].compute())

In [19]:
# kappa_win
# print(kappa_win.z_bins.keys())
# kappa_win.z_bins['kappa'][1]['window']
# kappa_win.Win.Win['cl']
# kappa_win.l
cl0[0].shape

(1000,)

In [ ]:
def kappa_map_gen(kappa, nside=nside):
    l = kappa.l
    l_bins = kappa.l_bins
    nu=2.*l+1.
    windows = {}
    masks = {}
    seen_indices = {}
    windows_one_indices = {}

    windows[0] = kappa.z_bins['kappa'][0]['window']
    masks[0] = windows[0]==hp.UNSEEN
    seen_indices[0] = np.where( windows[0]!=hp.UNSEEN )[0]
    windows_one_indices[0] = np.where( windows[0]==1.0 )[0]
    
    windows[1] = kappa.z_bins['galaxy'][0]['window']
    masks[1] = windows[1]==hp.UNSEEN
    seen_indices[1] = np.where( windows[1]!=hp.UNSEEN )[0]
    windows_one_indices[1] = np.where( windows[0]==1.0 )[0]
        
    
    cl_map = hp.synfast(cl0,nside=nside,new=False,pol=False)
    
    for i in range(2):
        cl_map[i]*=windows[i]
        cl_map[i][masks[i]] = hp.UNSEEN
    
    
    clp=hp.anafast(cl_map[0], lmax=max(l))
    
#     plt.plot(l, clp, '--', label='pseudo Cl from map')
#     plt.plot(lb, cl0_p[0], label='cl0_p')
#     plt.plot(l, cl0[0], label='cl0')
#     plt.loglog()
#     plt.legend()
#     plt.show()
    
    
#     hp.mollview(cl_map[0], title='kappa map (0th bin)')
#     plt.show()

    
    cl_sn = kappa_win.SN[corr_kk]
#     kappa_alm = hp.synalm(cl0,new=False) ## generate correlated kappa alm's
    kappa_alm = hp.map2alm(cl_map,pol=False, lmax=max(l)) ## generate correlated kappa alm's, with window/mask
    
#     for i in range(n_source_bins):
#         kappa_alm[i] += hp.synalm(cl_sn[:,i,i]) ## add the noise map
        
        
    trans_factor = [-np.sqrt( (li+2)*(li-1)/(li*(li+1))) for li in l   ] 
    trans_factor[0] = 0.
    trans_factor[1] = 0.
    
    gamma_alm = []
    for i in range(n_source_bins):
        gamma_alm.append(hp.sphtfunc.almxfl(kappa_alm[i], trans_factor))
    
    corr_config = {'min_sep':3., 'max_sep':600, 'nbins':10, 'sep_units':'arcmin'}
    
    k_map, g1_map, g2_map = hp.sphtfunc.alm2map( (kappa_alm[0],gamma_alm[0],kappa_alm[0]*0 ), nside=nside,pol=True  )
    g1_map[masks[0]] = hp.UNSEEN
    g2_map[masks[0]] = hp.UNSEEN
    
#     hp.mollview(g1_map, title='gamma 1 map (0th bin)')
#     plt.show()

#     hp.mollview(g2_map, title='gamma 2 map (0th bin)')
#     plt.show()
    
    
    ###################################
    ##   GG correlation
    ###################################
    theta, phi = hp.pix2ang(nside, seen_indices[0])
    ra = np.degrees(np.pi*2.-phi)
    dec = -np.degrees(theta-np.pi/2.)
    g1 = g1_map[seen_indices[0]]
    g2 = g2_map[seen_indices[0]]
    
    cat = treecorr.Catalog(ra=ra, dec=dec, g1=g1, g2=g2, ra_units='deg', dec_units='deg')
    gg = treecorr.GGCorrelation(**corr_config)
    gg.process(cat)
    
    
    ####################################
    ##  NN correlation (unit windows)
    ####################################
    theta_w, phi_w = hp.pix2ang(nside, windows_one_indices[0])
    ra_w = np.degrees(np.pi*2.-phi_w)
    dec_w = -np.degrees(theta_w-np.pi/2.)
    cat_win = treecorr.Catalog(ra=ra_w, dec=dec_w, ra_units='deg', dec_units='deg')
    cat_random = treecorr.Catalog(ra=ra, dec=dec, ra_units='deg', dec_units='deg')

    dd = treecorr.NNCorrelation(**corr_config)
    rr = treecorr.NNCorrelation(**corr_config)
    dr = treecorr.NNCorrelation(**corr_config)
    
    dd.process(cat_win)
    rr.process(cat_random)
    dr.process(cat_win, cat_random)
    
    xi,varxi = dd.calculateXi(rr,dr)
    
    
    #######################################
    ##  KK correlation
    #######################################
    k = cl_map[0][seen_indices[0]]
    cat_kk = cat = treecorr.Catalog(ra=ra, dec=dec, k=k, ra_units='deg', dec_units='deg')
    kk = treecorr.KKCorrelation(**corr_config)
    kk.process(cat_kk)
    
    
    return (gg.rnom, gg.xip, gg.xim), (dd.rnom, xi), (kk.rnom, kk.xi), clp
    
    


res = kappa_map_gen(kappa_win, nside=nside)

In [ ]:
print(res)
gg_res = res[0]
nn_res = res[1]
kk_res = res[2]

print(kappa_win.z_bins['kappa'][0]['window'])

In [ ]:
clp = res[3]

In [ ]:
th, w = WT_L.projected_correlation(l_cl=l, cl=cl0[0], m1_m2=(2,2))
th, wm = WT_L.projected_correlation(l_cl=l, cl=cl0[0], m1_m2=(2,-2))



th_p, w_p =  WT_L.projected_correlation(l_cl=lb, cl=cl0_p[0], m1_m2=(2,2))
th_p, w_p_sim = WT_L.projected_correlation(l_cl=l, cl=clp, m1_m2=(2,2))

plt.plot(th/d2r*60, w, label='cl')
plt.plot(th_p/d2r*60, w_p, label='cl_p')
plt.yscale('log')
plt.xscale('log')
plt.legend()
plt.xlabel('arcmin')
plt.ylabel('xi+')
plt.show()

In [ ]:
print("Healpix resolution with nside={} :  {} arcmin".format(nside, hp.nside2resol(nside, arcmin = True)))

plt.plot(gg_res[0], gg_res[1], label='sim')
plt.plot(th/d2r*60., w, '-', label='wigner (cl)')
plt.plot(th_p/d2r*60., w_p_sim/w_win, '-', label='wigner (cl_p)')

plt.title('xi+ (0,0)')
plt.xlabel('arcmin')
plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.show()

plt.plot(gg_res[0], gg_res[2])
plt.plot(th/d2r*60., wm, '-', label='wigner (cl)')
plt.title('xi- (0,0)')
plt.xlabel('arcmin')
plt.xscale('log')
plt.yscale('log')
plt.show()

plt.plot(kk_res[0], kk_res[1])
plt.title('<KK> (0,0)')
plt.xlabel('arcmin')
plt.xscale('log')
plt.yscale('log')
plt.show()

plt.plot(nn_res[0],nn_res[1])
plt.title('xi of window : (DD-2DR+RR)/RR')
plt.xlabel('arcmin')
plt.xscale('log')
plt.show()

In [ ]:
cl_win = hp.anafast(kappa_win.z_bins['kappa'][0]['window'], lmax=max(l))
th, w_win = WT_L.projected_correlation(l_cl=l, cl=cl_win,  m1_m2=(0,0))

plt.plot(th, w_win)
plt.loglog()
plt.show()

In [ ]:
n_sims = 100

results = [kappa_map_gen(kappa_win, nside=nside) for i in range(n_sims)]

In [ ]:
xip_samples = np.zeros( (10, n_sims) )
for i in range(n_sims):
    xip_samples[:,i] = results[i][0][1]
    
xip_cov = np.cov(xip_samples)

sep = results[0][0][0]
plt.plot(sep, np.diag(xip_cov))
plt.title('Cov of xip')
plt.xlabel('arcmin')
plt.xscale('log')
plt.yscale('log')
plt.show()